# Гомография (Проективная геометрия)

Будем делать демо приложения для дополненной реальности.

Вдохновлено: https://www.indiegogo.com/projects/lifeprint-photos-that-come-to-life-in-your-hands/

Для этого нам понадобится:
1. Найти ключевые точки на изоброжении, получить их дескрипторы и найти соответствия (матчи) между точками на двух изображениях
2. С помощью "сматченных" точек посчитать гомографию, чтобы можно было деформировать изображения

Максимально можно получить 50 баллов.

In [ ]:
import numpy as np
import cv2
import scipy
import skimage.color
import skimage.feature
import skimage.transform
import scipy.io as sio
from matplotlib import pyplot as plt

# 1. Поиск ключевых точек и матчей

Воспользуемся FAST детектором (он достаточно простой, погуглите про него информацию) для поиска ключевых точек, вместо рассмотренного на лекции детектора Харриса.

**Вопрос:** В чем основное отличие детектора FAST от Харриса? Является ли он быстрее?
    
**Ваш ответ:**

После того как мы нашли ключевые точки нам нужно их сматчить. Мы воспользуемся алгоритмом BRIEF (он достаточно простой, погуглите про него информацию), который присваивает бинарный дексриптор каждой ключевой точке. Для матчинга точек воспользуемся алгоритмом ближайших соседей на полученных дексрипторах. Расстояние между дексрипторами будем считать как расстояние Хэмминга.

**Вопрос:** Мы используем бинарные дексрипторы. В таком случае, в чем состоит преимущество расстояния Хэмминга в сравнении со стандартным Евклидовым расстоянием?
    
**Ваш ответ:**

Далее, ниже заранее реализованы набор функций, которые помогут вам в выполнении домашнего задания.

* `corner_detection` для поиска ключевых точек
* `computeBrief` для нахождения BRIEF дексриптора
* `briefMatch` матчим точки на основе их BRIEF дескрипторов
* `plotMatches` отрисовываем матчи

In [ ]:
PATCHWIDTH = 9

def briefMatch(desc1, desc2, ratio):
    matches = skimage.feature.match_descriptors(desc1, desc2, 'hamming', cross_check=True, max_ratio=ratio)
    return matches

def plotMatches(im1, im2, matches, locs1, locs2):
    fig, ax = plt.subplots(nrows=1, ncols=1)
    im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    plt.axis('off')
    skimage.feature.plot_matches(ax, im1, im2, locs1, locs2, matches, matches_color='r', only_matches=True)
    plt.show()
    return

def makeTestPattern(patchWidth, nbits):
    np.random.seed(0)
    compareX = patchWidth*patchWidth * np.random.random((nbits,1))
    compareX = np.floor(compareX).astype(int)
    np.random.seed(1)
    compareY = patchWidth*patchWidth * np.random.random((nbits,1))
    compareY = np.floor(compareY).astype(int)
    return (compareX, compareY)

def computePixel(img, idx1, idx2, width, center):
    halfWidth = width // 2
    col1 = idx1 % width - halfWidth
    row1 = idx1 // width - halfWidth
    col2 = idx2 % width - halfWidth
    row2 = idx2 // width - halfWidth
    return 1 if img[int(center[0]+row1)][int(center[1]+col1)] < img[int(center[0]+row2)][int(center[1]+col2)] else 0

def computeBrief(img, locs):
    patchWidth = 9
    nbits = 256
    compareX, compareY = makeTestPattern(patchWidth, nbits)
    m, n = img.shape

    halfWidth = patchWidth//2
    
    locs = np.array(list(filter(lambda x: halfWidth <= x[0] < m-halfWidth and halfWidth <= x[1] < n-halfWidth, locs)))
    desc = np.array([list(map(lambda x: computePixel(img, x[0], x[1], patchWidth, c), zip(compareX, compareY))) for c in locs])
    
    return desc, locs

def corner_detection(img, sigma):
    # fast method
    result_img = skimage.feature.corner_fast(img, n=PATCHWIDTH, threshold=sigma)
    locs = skimage.feature.corner_peaks(result_img, min_distance=1)
    return locs

### 1.1 (10 баллов)
Реализуйте функцию `matchPics`, которая принимает два изображения, и находит ключевые точки с соответствиями

In [ ]:
def matchPics(I1, I2, sigma=0.15, ratio=0.7):
    """
    Args:
        I1, I2: изображения в формате BGR
        ratio: параметр для BRIEF
        sigma: порог для FAST
    """

    # Ддя начала, делаем оба изображения черно-белыми с помощью opencv
    ##### your code here
    
    # Ищем ключевые точки на первом и втором изображении с помощью corner_detection()
    ##### your code here
    
    # Получаем дексрипторы для найденных ключевых точек для обоих изображений, используя computeBrief()
    ##### your code here

    # Получаем соответствия (matches) с помощью дескрипторов, используя briefMatch()
    ##### your code here
    
    return matches, locs1, locs2

In [ ]:
cv_cover = cv2.imread('data/cv_cover.jpg')
cv_desk = cv2.imread('data/cv_desk.png')

matches, locs1, locs2 = matchPics(cv_cover, cv_desk, sigma=0.15, ratio=0.7)
# рисуем матчи
plotMatches(cv_cover, cv_desk, matches, locs1, locs2)

### 1.2 (10 баллов)
Пофайнтюньте параметры sigma и ratio для функции `matchPics`. Опишите ваши наблюдения.

**Ответ:**

### 1.3 (10 баллов)
Дополнительно проверьте насколько BRIEF чувствителен к поворотам. Для этого поверните фотографию на x градусов, и посчитатей количество матчей между оригинальной и повернутой фотографией. Увеличивается / уменьшается число матчей? В чем причина? Проанализируйте для x = 0, 10, 20, 30, ..., 350

Для поворота фотографии воспользуйтесь `scipy.ndimage.rotate`

In [ ]:
##### your code here

# 2. Ищем гомографию

Гомография - это операция деформации, которая представляет собой преобразование координат пикселей с одной камеры (изображения, вида, ракурса) к другой камере (изображению, виду, ракурсу). При это делается фундаментальное удтверждение о том, что такое преобразование существует.

### 2.1 (10 баллов)

Необходимо найти такую 3x3 матрицу гомографии $\mathbf{H}$, что выполняется

$\mathbf{x}_{1} \equiv \mathbf{H x}_{2}$

Точки $\mathbf{x}_{1}$ и $\mathbf{x}_{2}$ имеют гомогенные координаты, т.е. $\left[x_{i}, y_{i}, z_{i}\right]^{T}$, что соответствует 2D точке $\left[\frac{x_{i}}{z_{i}},\frac{y_{i}}{z_{i}}
\right]^{T}$

Так как равенство $\mathbf{x}_{1} \equiv \mathbf{H x}_{2}$ выполняется вне зависимости от масштаба (scale), т.е. соответствует набору равенств $\mathbf{x}_{1} = \lambda\mathbf{H x}_{2}$, то искать $\mathbf{H}$ методом наименьших квадратов (Ordinary Least Squeares) не  получится, и нужно применить Direct Linear Transform (см. лекцию)

т.е. представить $\mathbf{x}_{1} \equiv \mathbf{H x}_{2}$ в виде $Ah=0$, где $h$ - вытянутая в вектор матрица $\mathbf{H}$ 

**Вопрос:** Сколько степений свободы (DOF) имеет $\mathbf{H}$?
    
**Ваш ответ:**


**Вопрос:** Сколько пар точек (матчей) необходимо чтобы вычислить $\mathbf{H}$? p.s. один матч даёт два уравнения
    
**Ваш ответ:**

In [ ]:
def computeH(x1, x2):
    """
    Args:
        x1 (numpy.ndarray): shape [Nx2]. N points from first image
        x2 (numpy.ndarray): shape [Nx2]. N points from second image
    Returns:
        H2to1 (numpy.ndarray): shape [3x3]. Homography matrix
    """
    N = x1.shape[0]
    A = np.zeros((2 * N, 3 * 3))
    
    # Заполните матрицу А используя x1 и x2 (см. лекцию, слайд 270)
    ##### your code here

    # Решению системы соответствует собственный вектор A^TA с наименьшим собственным числом (см. лекцию)
    _, _, eigen_vectors = np.linalg.svd(A)
    eigen_vector = eigen_vectors[-1, :]
    H2to1 = eigen_vector.reshape(3, 3)
    
    return H2to1

In [ ]:
# Проверка

x1 = np.array([
    [165, 246],
    [266, 305],
    [337, 228],
    [373, 64]
])

x2 = np.array([
    [510, 166],
    [364, 382],
    [387, 423],
    [419, 239],
])

h2to1_gt = np.array([[-1.38515339e-03, -1.44653089e-03,  9.84208461e-01],
                     [-2.21139337e-05, -6.60053015e-04,  1.76988735e-01],
                     [-2.32587684e-06, -3.89160630e-06,  2.06042265e-03]])

assert np.allclose(computeH(x1, x2), h2to1_gt)

### 2.2 (10 баллов)
Нормализация помогает избежать численной нестабильности и координаты лучше сначала пронормализовать, затем искать гомографию.

Делаем следующее:

1. Смещаем центроид координат к началу отсчета (0, 0)
2. Скалируем точки так, что максимальное расстояние до центра равно $\sqrt{2}$

Такое линейное преобразование можно записать в следующем виде:

$$\begin{array}{l}
\widetilde{\mathbf{x}}_{1}=\mathbf{T}_{1} \mathbf{x}_{1} \\
\widetilde{\mathbf{x}}_{2}=\mathbf{T}_{2} \mathbf{x}_{2}
\end{array}$$

where $\widetilde{\mathbf{x}}_{1}$ и $\widetilde{\mathbf{x}}_{2}$􏰂 нормализованные гомогенные координаты, полученные из $\mathbf{x}_{1}$ и $\mathbf{x}_{2}$, и матрицы $\mathbf{T}_{1}$ и $\mathbf{T}_{2}$ размера 3x3.

`computeH` выдает гомографию, которая удовлетворяет:

$$\widetilde{\mathbf{x}}_{1}=\mathbf{H} \widetilde{\mathbf{x}}_{2}$$

Заменяя $\widetilde{\mathbf{x}}_{1}$􏰂 и $\widetilde{\mathbf{x}}_{2}$ на $\mathbf{T}_{1}\mathbf{x}_{1}$ и $\mathbf{T}_{2}\mathbf{x}_{2}$, получаем:

$$\begin{array}{l}
\mathbf{T}_{1} \mathbf{x}_{1}=\mathbf{H} \mathbf{T}_{2} \mathbf{x}_{2} \\
\mathbf{x}_{1}=\mathbf{T}_{1}^{-1} \mathbf{H} \mathbf{T}_{2} \mathbf{x}_{2}
\end{array}$$

In [ ]:
def computeH_norm(x1, x2):
    """
    Args:
        x1 (numpy.ndarray): shape [Nx2], N точек из первого изображения
        x2 (numpy.ndarray): shape [Nx2], N точек из второго изображения
    Returns:
        H2to1 (numpy.ndarray): shape [3x3]. Homography matrix
    """
    x1_ = x1.astype(float)
    x2_ = x2.astype(float)
    num_points = x1.shape[0]

    # Вычислите два центроида для точек, один для x1, другой для x2
    ##### your code here

    # Переместите центр координат к центроидам
    ##### your code here
    # x1_ = ...
    # x2_ = ...

    # Нормализуйте точки x1 и x2 независимо, так чтобы наибольшая дистанция до сооответствуюшего
    # центра координат составляла sqrt(2).
    # Для этого достаточно убедиться что ни одна из координат не является по модулю больше 1.
    # Нормализуйте каждую из координат (x, y) для каждого из набора точек (x1, x2) по-отдельности
    ##### your code here
    # x1_[:, 0] = ...
    # x1_[:, 1] = ...
    # x2_[:, 0] = ...
    # x2_[:, 1] = ...

    # Выпишем чему равно T1 и T2
    T1 = np.dot(np.c_[x1_, np.ones(num_points)].T, np.linalg.pinv(np.c_[x1, np.ones(num_points)].T))
    T2 = np.dot(np.c_[x2_, np.ones(num_points)].T, np.linalg.pinv(np.c_[x2, np.ones(num_points)].T))

    # Ищем гомографию (см. формулу выше)
    H2to1 = computeH(x1_, x2_)
    H2to1 = np.linalg.pinv(T1).dot(H2to1).dot(T2)

    return H2to1

In [ ]:
# Проверка

x1 = np.array([
    [165, 246],
    [266, 305],
    [337, 228],
    [373, 64]
])

x2 = np.array([
    [510, 166],
    [364, 382],
    [387, 423],
    [419, 239],
])

computeH_norm(x1, x2)

h2to1_gt = np.array([[-1.46924418e+00, -1.53434783e+00,  1.04395843e+03],
                     [-2.34564407e-02, -7.00123940e-01,  1.87733482e+02],
                     [-2.46707768e-03, -4.12786045e-03,  2.18550812e+00]])

assert np.allclose(computeH_norm(x1, x2), h2to1_gt)

### 2.3 RANSAC

RANSAC - метод поиска параметров модели на основе случайных подвыборок данных, устойчивый к шумам в данных.

Это идеально соответствует нашему случаю, мы имеем матчей больше чем необходимо, чтобы найти гомографию. Однако не все матчи правильные (имеем выбросы, шум в данных), что может плохо повлиять на подсчет матрицы гомографии.

Ниже представлена уже готовая реализация `computeH_ransac`, внимательно ее изучите.

In [ ]:
def computeH_ransac(matches, locs1, locs2, max_iters=1000, inlier_tol=1):
    """
    Найти наилучшую гомографию на основе матчей
    
    Args:
        matches: (numpy.ndarray): shape [Mx2], M пар индексов (i, j) где i-индекс точки из locs1, j-индекс точки из locs2
        locs1: (numpy.ndarray): shape [Nx2], N точек из первого изображения
        locs2: (numpy.ndarray): shape [Nx2], N точек из второго изображения
        max_iters (int): число итераций для RANSAC
        inlier_tol (float): порогове значение, при котором точку считаеем inlier (не-выбросом)
    Returns:
        H2to1 (numpy.ndarray): shape [3x3]. Homography matrix
    """
    locs1 = locs1[matches[:, 0]]
    locs2 = locs2[matches[:, 1]]
    num_points = locs1.shape[0]
    max_inliers = 0
    bestH2to1 = np.empty([3, 3], dtype=float)
    inliers = np.empty([num_points, 1], dtype=int)

    for i in range(max_iters):
        points = np.sort(np.random.randint(low=0, high=num_points, size=4))
        p1 = locs1[points]
        p2 = locs2[points]
        try:
            # ищем гомографию
            H2to1 = computeH_norm(p1, p2)
            
            # добавляем третью размерность (гомогенный вид)
            x1_hom = np.vstack((locs1.T, np.ones((1, num_points))))
            x2_hom = np.vstack((locs2.T, np.ones((1, num_points))))
            
            # деформируем точки x2
            new_p1 = H2to1.dot(x2_hom)
            
            # переходим от гомогенных координат к 2D координатам
            new_p1 = new_p1 / new_p1[-1, :]
            
            # сравниваем деформированные x2 и настоящие точки x1 
            error = new_p1 - x1_hom
            dist = np.linalg.norm(error, axis=0)
            
            # если число inliers оказалось больше чем до этого, обновляем лучшего кандидата для матрицы гомографии
            consensus = np.where(dist <= inlier_tol)
            inliers[consensus] = 1
            not_consensus = np.where(dist > inlier_tol)
            inliers[not_consensus] = 0
            num_inliers = consensus[0].shape[0]
            if num_inliers > max_inliers:
                max_inliers = num_inliers
                bestH2to1 = H2to1
        except:
            pass

    return bestH2to1

### 2.4 Всё вместе (10 баллов)

Необходимо деформировать шаблонную картинку в плоскость сцены.

Для этого у нас есть три изображения.

* Первое (img) - сцена.
* Второе (cv_cover) - шаблон обложки книги, которая находится в сцене и мы хотим ее заменить.
* Третье (template) - шаблон обложки книги, которую мы вставим в сцену.

Заметьте, что мы считаем гомографию от картинки к шаблону.
Для того, чтобы сделать наоборот, деформировать шаблон, чтобы выписать его в картинку, необходимо получить матрицу обратную к найденной.

In [ ]:
def compositeH(H2to1, template, img, cv_cover):

    # Необходимо изменить размер template, так чтобы он был равен к cv_cover. Воспользуйтесь cv2.resize
    ##### your code here

    # деформируем template используя матрицу гомографии
    template = cv2.warpPerspective(template.swapaxes(0, 1), np.linalg.inv(H2to1), (img.shape[0], img.shape[1])).swapaxes(0, 1)
    
    # объединяем результат (блендим изображения: template и img)
    # p.s. ключевое слово - маска
    ##### your code here
    
    return compositeimg

### Смотрим результат

In [ ]:
cv_cover = cv2.imread('data/cv_cover.jpg')
cv_desk = cv2.imread('data/cv_desk.png')
hp_cover = cv2.imread('data/hp_cover.jpg')

matches, locs1, locs2 = matchPics(cv_cover, cv_desk)
bestH2to1 = computeH_ransac(matches, locs1, locs2, max_iters=100)
composite_img = compositeH(bestH2to1, hp_cover, cv_desk, cv_cover)
plt.imshow(cv2.cvtColor(composite_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

Должно получиться примерно следующее:
![](references/ref.png)

**Вопрос:** Что было бы если мы не изменяли размер template в `compositeH`? Попробуйте запустить без ресайза. Объясните получившийся результат.
    
**Ваш ответ:**


# 3. Дополненная реальность

Вместо подставления одного изображения в другое, давайте сделаем это для видео. Всё тоже самое, ведь видео это набор кадров.

In [ ]:
def loadVid(path):
    """
    Args:
        path: путь к видео
    Returns:
        frames (numpy.array): shape [TxHxWx3]. Раскадрованное видео
    """
    cap = cv2.VideoCapture(path)
    
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")

    i = 0
    
    while(cap.isOpened()):
        i += 1
        ret, frame = cap.read()
        if ret == True:
            if i == 1:
                frames = frame[np.newaxis, ...]
            else:
                frame = frame[np.newaxis, ...]
                frames = np.vstack([frames, frame])
                frames = np.squeeze(frames)
        else: 
            break
    
    cap.release()

    return frames

In [ ]:
%%time
cv_cover = cv2.imread('data/cv_cover.jpg')
ar_source = loadVid('data/ar_source.mov')
book = loadVid('data/book.mov')

In [ ]:
%%time
# Процесс может занять достаточно длительное время

# Обрабатываем видео кадр за кадром
cap = cv2.VideoWriter('ar.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 15.0, (book.shape[0], book.shape[1]))
for frame_num in range(ar_source.shape[0]):
    frame_source = ar_source[frame_num]
    frame_book = book[frame_num]
    matches, locs1, locs2 = matchPics(cv_cover, frame_book)
    bestH2to1 = computeH_ransac(matches, locs1, locs2, max_iters=300)
    
    # обрезаем видео кадр, чтобы не попали черные полосы
    frame_source = frame_source[48:-48, 145:495]
    
    composite_img = compositeH(bestH2to1, frame_source, frame_book, cv_cover)
    
    cap.write(composite_img)
cap.release()

Попробуйте такое проделать с вашими данными!